# Add annotations

This notebook takes the dataframe with information about module composition and their labels and adds additional annotations including:

1. Which gene is contained within the modules (both gene id and gene name)
2. KEGG pathways that genes are found in
3. GO pathways genes are found in
4. Regulon/operon genes are found in

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from core_acc_modules import utils, paths

random.seed(1)

/home/alexandra/anaconda3/envs/core_acc/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# User param
method = "affinity"

In [3]:
# Import module labels
pao1_module_label_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pao1_gene_module_labels_{method}.tsv"
)
pa14_module_label_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pa14_gene_module_labels_{method}.tsv"
)

pao1_module_labels = pd.read_csv(
    pao1_module_label_filename, sep="\t", index_col=0, header=0
)
pa14_module_labels = pd.read_csv(
    pa14_module_label_filename, sep="\t", index_col=0, header=0
)

In [4]:
# Import gene memberships
pao1_membership_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pao1_modules_{method}.tsv"
)
pa14_membership_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pa14_modules_{method}.tsv"
)

pao1_membership = pd.read_csv(pao1_membership_filename, sep="\t", index_col=0, header=0)
pa14_membership = pd.read_csv(pa14_membership_filename, sep="\t", index_col=0, header=0)

In [5]:
# Import gene metadata
pao1_gene_annot_filename = paths.GENE_PAO1_ANNOT
pa14_gene_annot_filename = paths.GENE_PA14_ANNOT

pao1_gene_annot = pd.read_csv(pao1_gene_annot_filename, index_col=0, header=0)
pa14_gene_annot = pd.read_csv(pa14_gene_annot_filename, index_col=0, header=0)

In [6]:
pao1_gene_annot = pao1_gene_annot["Name"].to_frame("gene name")
pa14_gene_annot = pa14_gene_annot["Name"].to_frame("gene name")

## Add module labels

In [7]:
# Add module labels
pao1_gene_module_labels = pao1_membership.merge(
    pao1_module_labels, left_on="module id", right_index=True
)
pa14_gene_module_labels = pa14_membership.merge(
    pa14_module_labels, left_on="module id", right_index=True
)

## Add gene names

In [8]:
# Add gene names
pao1_gene_module_labels = pao1_gene_module_labels.merge(
    pao1_gene_annot, left_index=True, right_index=True
)
pa14_gene_module_labels = pa14_gene_module_labels.merge(
    pa14_gene_annot, left_index=True, right_index=True
)

In [9]:
print(pao1_gene_module_labels.shape)
pao1_gene_module_labels.head()

(5563, 10)


,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name
PA0001,415,12,0,5349,202,inf,1.0,mixed,12,dnaA
PA0006,415,12,0,5349,202,inf,1.0,mixed,12,NaN
PA1440,415,12,0,5349,202,inf,1.0,mixed,12,NaN
PA1544,415,12,0,5349,202,inf,1.0,mixed,12,anr
PA1741,415,12,0,5349,202,inf,1.0,mixed,12,NaN


In [10]:
print(pa14_gene_module_labels.shape)
pa14_gene_module_labels.head()

(5887, 10)


,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name
PA14_55610,3,14,1,5343,533,1.396594,1.0,mixed,15,dnaE2
PA14_55140,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN
PA14_55130,3,14,1,5343,533,1.396594,1.0,mixed,15,gloA2
PA14_54120,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN
PA14_51650,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN


## Add core/accessory annotations

In [11]:
# Read in expression data
pao1_expression_filename = paths.PAO1_COMPENDIUM
pa14_expression_filename = paths.PA14_COMPENDIUM

pao1_expression = pd.read_csv(pao1_expression_filename, sep="\t", index_col=0, header=0)
pa14_expression = pd.read_csv(pa14_expression_filename, sep="\t", index_col=0, header=0)

In [12]:
core_acc_dict = utils.get_my_core_acc_genes(
    pao1_gene_annot_filename, pa14_gene_annot_filename, pao1_expression, pa14_expression
)

Number of PAO1 core genes: 5366
Number of PA14 core genes: 5363
Number of PAO1 core genes in my dataset: 5361
Number of PA14 core genes in my dataset: 5357
Number of PAO1-specific genes: 202
Number of PA14-specific genes: 534


In [13]:
pao1_core = core_acc_dict["core_pao1"]
pa14_core = core_acc_dict["core_pa14"]
pao1_acc = core_acc_dict["acc_pao1"]
pa14_acc = core_acc_dict["acc_pa14"]

In [14]:
pao1_gene_module_labels.loc[pao1_core, "core/acc"] = "core"
pao1_gene_module_labels.loc[pao1_acc, "core/acc"] = "acc"

In [15]:
pa14_acc_shared = set(pa14_acc).intersection(pa14_gene_module_labels.index)
pa14_gene_module_labels.loc[pa14_core, "core/acc"] = "core"
pa14_gene_module_labels.loc[pa14_acc_shared, "core/acc"] = "acc"

In [16]:
pao1_gene_module_labels.head()

,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name,core/acc
PA0001,415,12,0,5349,202,inf,1.0,mixed,12,dnaA,core
PA0006,415,12,0,5349,202,inf,1.0,mixed,12,NaN,core
PA1440,415,12,0,5349,202,inf,1.0,mixed,12,NaN,core
PA1544,415,12,0,5349,202,inf,1.0,mixed,12,anr,core
PA1741,415,12,0,5349,202,inf,1.0,mixed,12,NaN,core


In [17]:
pa14_gene_module_labels.head()

,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name,core/acc
PA14_55610,3,14,1,5343,533,1.396594,1.0,mixed,15,dnaE2,core
PA14_55140,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN,core
PA14_55130,3,14,1,5343,533,1.396594,1.0,mixed,15,gloA2,core
PA14_54120,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN,core
PA14_51650,3,14,1,5343,533,1.396594,1.0,mixed,15,NaN,acc


## Add KEGG pathways

For each pathway, what genes are contained in it

In [18]:
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

pao1_pathways = pd.read_csv(pao1_pathway_filename, sep="\t", index_col=0, header=None)

In [19]:
pao1_pathways[2] = pao1_pathways[2].str.split(";").apply(set)
pao1_pathways.index = pao1_pathways.index.str.split(" - ").str[0]
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA1999, PA2000, PA4785, PA3589, PA2001, PA255..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA5349, PA4994, PA3013, PA3014, PA2001, PA087..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA3426, PA1821, PA4899, PA1027, PA1748, PA301..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA2081, PA2080, PA3366, PA3014, PA2001, PA434..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA3633, PA4785, PA3803, PA3627, PA3650, PA358..."


In [20]:
gene_to_pathways_df = pd.DataFrame(
    index=pao1_gene_module_labels.index, columns=list(pao1_pathways.index)
)

In [21]:
%%time
for gene in gene_to_pathways_df.index:
    gene_to_pathways_df.loc[gene] = [
        gene in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
    ]

CPU times: user 2min 27s, sys: 46.2 ms, total: 2min 27s
Wall time: 2min 27s


## Add operon

For each operon, what genes are contained in it

NOTE: This code takes a while to run so for now its commented out

In [22]:
pao1_operon_filename = "https://raw.githubusercontent.com/greenelab/core-accessory-interactome/6635c0e357c0172c2cebd0368648030e0ee4beaf/data/metadata/operons_format.csv"

pao1_operons = pd.read_csv(pao1_operon_filename, index_col=0, header=0)

In [23]:
pao1_operons.head()

,Length,Genes
OperonID,,
12029,4,PA0001;PA0002;PA0003;PA0004
12030,2,PA0005;PA0006
12031,2,PA0008;PA0009
12032,3,PA0016;PA0017;PA0018
12033,2,PA0021;PA0022


In [24]:
pao1_operons["Genes"] = pao1_operons["Genes"].str.split(";").apply(set)
pao1_operons.head()

,Length,Genes
OperonID,,
12029,4,"{PA0001, PA0003, PA0004, PA0002}"
12030,2,"{PA0006, PA0005}"
12031,2,"{PA0009, PA0008}"
12032,3,"{PA0016, PA0018, PA0017}"
12033,2,"{PA0022, PA0021}"


In [25]:
# Remove operons with a single gene
pao1_operons = pao1_operons[pao1_operons["Genes"].apply(len) > 1]

In [26]:
gene_to_operons_df = pd.DataFrame(
    index=pao1_gene_module_labels.index, columns=list(pao1_operons.index)
)

In [27]:
%%time
for gene in gene_to_operons_df.index:
    gene_to_operons_df.loc[gene] = [
        gene in pao1_operons.loc[operon, "Genes"] for operon in pao1_operons.index
    ]

CPU times: user 18min 56s, sys: 1min 54s, total: 20min 51s
Wall time: 20min 53s


In [28]:
# Add operons to pathway annotations
pao1_gene_annot = gene_to_pathways_df.merge(
    gene_to_operons_df, left_index=True, right_index=True, how="outer"
)

In [29]:
print(pao1_gene_annot.shape)
pao1_gene_annot.head()

(5563, 1308)


,KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,KEGG-Pathway-pae00071: Fatty acid degradation,KEGG-Pathway-pae00903: Limonene and pinene degradation,KEGG-Pathway-pae00380: Tryptophan metabolism,KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,KEGG-Pathway-pae00660: C5-Branched dibasic acid metabolism,"KEGG-Pathway-pae00260: Glycine, serine and threonine metabolism",KEGG-Pathway-pae00780: Biotin metabolism,KEGG-Pathway-pae02060: Phosphotransferase system (PTS),KEGG-Pathway-pae00364: Fluorobenzoate degradation,...,13158,13159,13160,13161,13162,13163,13164,13165,13166,13167
PA0001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA0006,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1544,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1741,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Add regulon

For each regulon, what genes are contained in it

In [30]:
pao1_regulon_filename = "https://raw.githubusercontent.com/greenelab/core-accessory-interactome/6635c0e357c0172c2cebd0368648030e0ee4beaf/data/metadata/regulons_format.csv"

pao1_regulons = pd.read_csv(pao1_regulon_filename, index_col=0, header=0)

In [31]:
pao1_regulons["Genes"] = pao1_regulons["Genes"].str.split(";").apply(set)

In [32]:
gene_to_regulons_df = pd.DataFrame(
    index=pao1_gene_module_labels.index, columns=list(pao1_regulons.index)
)

In [33]:
%%time
for gene in gene_to_regulons_df.index:
    gene_to_regulons_df.loc[gene] = [
        gene in pao1_regulons.loc[regulon, "Genes"] for regulon in pao1_regulons.index
    ]

CPU times: user 14.6 s, sys: 105 µs, total: 14.6 s
Wall time: 14.6 s


In [34]:
# Add regulons to other annotations
pao1_gene_annot = pao1_gene_annot.merge(
    gene_to_regulons_df, left_index=True, right_index=True, how="outer"
)

In [35]:
print(pao1_gene_annot.shape)
pao1_gene_annot.head()

(5563, 1325)


,KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,KEGG-Pathway-pae00071: Fatty acid degradation,KEGG-Pathway-pae00903: Limonene and pinene degradation,KEGG-Pathway-pae00380: Tryptophan metabolism,KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,KEGG-Pathway-pae00660: C5-Branched dibasic acid metabolism,"KEGG-Pathway-pae00260: Glycine, serine and threonine metabolism",KEGG-Pathway-pae00780: Biotin metabolism,KEGG-Pathway-pae02060: Phosphotransferase system (PTS),KEGG-Pathway-pae00364: Fluorobenzoate degradation,...,PqsR_regulon,QscR_regulon,VreI_regulon,Zur_regulon,Anr_short_list,PhoB_short_list,AlgU_short_list,LasR_short_list,RhlR_short_list,PqsR_short_list
PA0001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA0006,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1544,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA1741,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Map pathway, operon, regulon to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14

In [36]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [37]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

In [38]:
# Map PA14 gene ids
pao1_pa14_gene_annot = pao1_gene_annot.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)

In [39]:
pao1_pa14_gene_annot.head()

,KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,KEGG-Pathway-pae00071: Fatty acid degradation,KEGG-Pathway-pae00903: Limonene and pinene degradation,KEGG-Pathway-pae00380: Tryptophan metabolism,KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,KEGG-Pathway-pae00660: C5-Branched dibasic acid metabolism,"KEGG-Pathway-pae00260: Glycine, serine and threonine metabolism",KEGG-Pathway-pae00780: Biotin metabolism,KEGG-Pathway-pae02060: Phosphotransferase system (PTS),KEGG-Pathway-pae00364: Fluorobenzoate degradation,...,QscR_regulon,VreI_regulon,Zur_regulon,Anr_short_list,PhoB_short_list,AlgU_short_list,LasR_short_list,RhlR_short_list,PqsR_short_list,PA14_ID
PA0001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,PA14_00010
PA0006,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,PA14_00070
PA1440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,PA14_45840
PA1544,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,PA14_44490
PA1741,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,PA14_42020


In [40]:
# Reset index to PA14 gene ids
pa14_gene_annot = pao1_pa14_gene_annot.set_index("PA14_ID")
print(pa14_gene_annot.shape)
pa14_gene_annot.head()

(5563, 1325)


,KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,KEGG-Pathway-pae00071: Fatty acid degradation,KEGG-Pathway-pae00903: Limonene and pinene degradation,KEGG-Pathway-pae00380: Tryptophan metabolism,KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,KEGG-Pathway-pae00660: C5-Branched dibasic acid metabolism,"KEGG-Pathway-pae00260: Glycine, serine and threonine metabolism",KEGG-Pathway-pae00780: Biotin metabolism,KEGG-Pathway-pae02060: Phosphotransferase system (PTS),KEGG-Pathway-pae00364: Fluorobenzoate degradation,...,PqsR_regulon,QscR_regulon,VreI_regulon,Zur_regulon,Anr_short_list,PhoB_short_list,AlgU_short_list,LasR_short_list,RhlR_short_list,PqsR_short_list
PA14_ID,,,,,,,,,,,,,,,,,,,,,
PA14_00010,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA14_00070,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA14_45840,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA14_44490,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
PA14_42020,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
# Merge annotations with module labels
pao1_gene_summary = pao1_gene_module_labels.merge(
    pao1_gene_annot, left_index=True, right_index=True, how="left"
)
pa14_gene_summary = pa14_gene_module_labels.merge(
    pa14_gene_annot, left_index=True, right_index=True, how="left"
)

In [42]:
print(pao1_gene_summary.shape)
pao1_gene_summary.head()

(5563, 1336)


,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name,...,PqsR_regulon,QscR_regulon,VreI_regulon,Zur_regulon,Anr_short_list,PhoB_short_list,AlgU_short_list,LasR_short_list,RhlR_short_list,PqsR_short_list
PA0001,415,12,0,5349,202,inf,1.0,mixed,12,dnaA,...,False,False,False,False,False,False,False,False,False,False
PA0006,415,12,0,5349,202,inf,1.0,mixed,12,NaN,...,False,False,False,False,False,False,False,False,False,False
PA1440,415,12,0,5349,202,inf,1.0,mixed,12,NaN,...,False,False,False,False,False,False,False,False,False,False
PA1544,415,12,0,5349,202,inf,1.0,mixed,12,anr,...,False,False,False,False,False,False,False,False,False,False
PA1741,415,12,0,5349,202,inf,1.0,mixed,12,NaN,...,False,False,False,False,False,False,False,False,False,False


In [43]:
print(pa14_gene_summary.shape)
pa14_gene_summary.head()

(5890, 1336)


,module id,num core in module,num acc in module,num core outside module,num acc outside module,odds ratio,p-value,module label,size,gene name,...,PqsR_regulon,QscR_regulon,VreI_regulon,Zur_regulon,Anr_short_list,PhoB_short_list,AlgU_short_list,LasR_short_list,RhlR_short_list,PqsR_short_list
PA14_00010,537,8,0,5349,534,inf,1.000000,mixed,8,dnaA,...,False,False,False,False,False,False,False,False,False,False
PA14_00020,537,8,0,5349,534,inf,1.000000,mixed,8,dnaN,...,False,False,False,False,False,False,False,False,False,False
PA14_00030,537,8,0,5349,534,inf,1.000000,mixed,8,recF,...,False,False,False,False,False,False,False,False,False,False
PA14_00050,252,7,0,5350,534,inf,1.000000,mixed,7,gyrB,...,False,False,False,False,False,False,False,False,False,False
PA14_00060,276,23,1,5334,533,2.298275,0.719108,mixed,24,NaN,...,False,False,False,False,False,False,False,False,False,False


In [44]:
# Drop duplicates
pa14_gene_summary = pa14_gene_summary[~pa14_gene_summary.index.duplicated(keep=False)]

In [45]:
# Save
pao1_gene_summary.to_csv(
    os.path.join(paths.LOCAL_DATA_DIR, f"pao1_gene_module_annotated_{method}.tsv"),
    sep="\t",
)
pa14_gene_summary.to_csv(
    os.path.join(paths.LOCAL_DATA_DIR, f"pa14_gene_module_annotated_{method}.tsv"),
    sep="\t",
)